In [9]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models, similarities, matutils

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
cleaned_csv_location = '../cleaned_tweets.csv'
    
def read_cleaned_df():    
    return pd.read_csv(cleaned_csv_location, index_col='id')

In [10]:
russia_df = read_cleaned_df()

non_rt_tweets = russia_df[~russia_df['text'].str.contains('RT', na=False)]['text']

In [12]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=10)
vectorized_tweets = tfidf.fit_transform(non_rt_tweets)

#pd.DataFrame(vectorized_tweets.todense(), 
#             columns=tfidf.get_feature_names()
#            ).head()

tfidf_corpus = matutils.Sparse2Corpus(vectorized_tweets.transpose())

# Row indices
id2word = dict((v, k) for k, v in tfidf.vocabulary_.items())

# This is a hack for Python 3!
id2word = corpora.Dictionary.from_corpus(tfidf_corpus, 
                                         id2word=id2word)




In [ ]:
lsi = models.LsiModel(tfidf_corpus, id2word=id2word, num_topics=300)

In [ ]:
# Retrieve vectors for the original tfidf corpus in the LSI space ("transform" in sklearn)
lsi_corpus = lsi[tfidf_corpus]

# Dump the resulting document vectors into a list so we can take a look
doc_vecs = [doc for doc in lsi_corpus]
doc_vecs[0]